In [32]:
from PIL import Image
from keras.models import load_model
from numpy import asarray
from numpy import expand_dims
from keras_facenet import FaceNet

import numpy as np
import pickle
import cv2

In [33]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [35]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    luuKetQua = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(luuKetQua)>0:
        x1, y1, width, height = luuKetQua[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # chuyển đổi openCV sang PIL
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)

    face = expand_dims(face, axis=0)
    signature = MyFaceNet.embeddings(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (20,40),cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (255,255,255), 2)
        
    cv2.imshow('res',gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 64ms/step
